#Project 1: Modeling

In [3]:
#importing packages
import pandas as pd
import seaborn as sns
#numpy necessary?
#also need to import scikit ML packages for step 6 onwards, and maybe for preprocessing too

url = "https://raw.githubusercontent.com/zsturk/Dat402_Proj1/refs/heads/Step-1/ObesityDataSet_raw_and_data_sinthetic.csv"
df = pd.read_csv(url, sep = ",")

Describe dataset here and describe our goal in modeling, etc

##Cleaning data

In [ ]:
#data cleaning and wrangling etc

##Exploratory Data Analysis

In [ ]:
#some stuff

In [ ]:
#other stuff

##Splitting the dataset, scaling etc

In [ ]:
#stuff

In [ ]:
#probably more stuff

##Model 1: abc

In [ ]:
#modeling...
#prediction

##Model 2: xyz

In [ ]:
#modeling...
#prediction

##Conclusion